# Stochastic processes and reproducibility

## Generating random numbers

In [18]:
import agentpy as ap
import numpy as np
import random

To illustrate, let us define a model that generates a list of ten pseudo-random numbers:

In [10]:
class RandomModel(ap.Model):
    
    def setup(self):
        self.random_numbers = [self.random.randint(0, 9) for _ in range(10)]
        print(f"Model {self.p.n} generated the numbers {self.random_numbers}")

Now if we run this model multiple times, we will get a different series of numbers:

In [11]:
for i in range(2):
    parameters = {'steps':0, 'n':i}
    model = RandomModel(parameters)
    results = model.run(display=False)

Model 0 generated the numbers [9, 4, 7, 7, 5, 5, 8, 2, 9, 6]
Model 1 generated the numbers [2, 9, 9, 1, 1, 1, 6, 9, 8, 9]


In [12]:
for i in range(2):
    parameters = {'seed':1, 'steps':0, 'n':i}
    model = RandomModel(parameters)
    model.run(display=False)

Model 0 generated the numbers [1, 4, 1, 7, 7, 7, 6, 3, 1, 7]
Model 1 generated the numbers [1, 4, 1, 7, 7, 7, 6, 3, 1, 7]


## Using multiple generators

In [19]:
class RandomAgent2(ap.Agent):
    
    def setup(self):
        seed = self.model.random.getrandbits(128) # Seed from model
        self.random = random.Random(seed)  # Create agent generator
        self.random_numbers = [self.random.randint(0, 9) for _ in range(10)]
        print(f"{self} generated the numbers {self.random_numbers}")
        
class RandomModel2(ap.Model):
    
    def setup(self):
        print(f"Model {i}:")
        self.agents = ap.AgentList(self, 2, RandomAgent2)
        print()

for i in range(2):
    parameters = {'seed': 1, 'steps': 0}
    model = RandomModel2(parameters)
    results = model.run(display=False)

Model 0:
RandomAgent2 (Obj 1) generated the numbers [8, 1, 4, 6, 6, 3, 4, 3, 5, 1]
RandomAgent2 (Obj 2) generated the numbers [9, 7, 8, 8, 0, 5, 3, 0, 2, 6]

Model 1:
RandomAgent2 (Obj 1) generated the numbers [8, 1, 4, 6, 6, 3, 4, 3, 5, 1]
RandomAgent2 (Obj 2) generated the numbers [9, 7, 8, 8, 0, 5, 3, 0, 2, 6]



Alternatively, we could also have each agent start from the same seed:

In [22]:
class RandomAgent3(ap.Agent):
    
    def setup(self):
        self.random = random.Random(self.p.agent_seed)
        self.random_numbers = [self.random.randint(0, 9) for _ in range(10)]
        print(f"{self} generated the numbers {self.random_numbers}")
        
class RandomModel3(ap.Model):
    
    def setup(self):
        print(f"\nModel {i}:")
        self.agents = ap.AgentList(self, 2, RandomAgent3)
        
for i in range(2):
    parameters = {'agent_seed': 1, 'steps':0, 'n':i}
    model = RandomModel3(parameters)
    results = model.run(display=False)


Model 0:
RandomAgent3 (Obj 1) generated the numbers [2, 9, 1, 4, 1, 7, 7, 7, 6, 3]
RandomAgent3 (Obj 2) generated the numbers [2, 9, 1, 4, 1, 7, 7, 7, 6, 3]

Model 1:
RandomAgent3 (Obj 1) generated the numbers [2, 9, 1, 4, 1, 7, 7, 7, 6, 3]
RandomAgent3 (Obj 2) generated the numbers [2, 9, 1, 4, 1, 7, 7, 7, 6, 3]


## Modeling stochastic processes

This section presents some stochastic operations that are often used in agent-based models. To start, we prepare a generic model with ten agents:

In [23]:
model = ap.Model()
agents = ap.AgentList(model, 10)
agents

AgentList (10 objects)

If we look at the agent's ids, we see that they have been created in order:

In [24]:
agents.id

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [25]:
agents.shuffle().id

[8, 10, 5, 6, 9, 4, 7, 2, 3, 1]

In [46]:
agents.random(3).id

[7, 5, 9]

With repetition/replacement:

In [47]:
agents.random(20, replace=True).id

[6, 7, 9, 9, 5, 2, 1, 8, 2, 8, 2, 6, 8, 10, 3, 10, 7, 7, 9, 1]

## Multi-run experiments

If we want to run a model multiple times,
there are different ways to define the random seeds.
If we pass no seed at all, every run will be different.
If we pass a seed as a parameter, every iteration with that parameter will have the same results.

In [48]:
class Model(ap.Model):
    def setup(self):
        self.report('x', self.model.random.random())

parameters = {'steps': 0, 'seed': ap.Values(1, 1, 2)}
sample = ap.Sample(parameters)

In [49]:
exp = ap.Experiment(Model, sample, iterations=2)
results = exp.run()
results.arrange_reporters()

Scheduled runs: 6
Completed: 6, estimated time remaining: 0:00:00
Experiment finished
Run time: 0:00:00.027598


,sample_id,iteration,x,seed
0,0,0,0.763775,1
1,0,1,0.763775,1
2,1,0,0.763775,1
3,1,1,0.763775,1
4,2,0,0.056551,2
5,2,1,0.056551,2


Alternatively, we can initialize the experiment with `random=True` to use the seed parameter as a seed to generate new seeds for each iteration with that parameter combination.

In [50]:
sample = ap.Sample(parameters)
exp = ap.Experiment(Model, sample, iterations=2, random=True)
results = exp.run()
results.arrange_reporters()

Scheduled runs: 6
Completed: 6, estimated time remaining: 0:00:00
Experiment finished
Run time: 0:00:00.031210


,sample_id,iteration,x,seed
0,0,0,0.035423,1
1,0,1,0.083635,1
2,1,0,0.035423,1
3,1,1,0.083635,1
4,2,0,0.134744,2
5,2,1,0.502731,2


## Further reading

- Random number generation in Python:  
  https://realpython.com/python-random/
- Random sampling in Numpy:  
  https://numpy.org/devdocs/reference/random/index.html
- Stochasticity in agent-based models:  
  http://www2.econ.iastate.edu/tesfatsi/ace.htm#Stochasticity
- Pseudo-random number generators:  
  https://en.wikipedia.org/wiki/Pseudorandom_number_generator
- What is random:  
  https://www.youtube.com/watch?v=9rIy0xY99a0